# Mail invite to list
Send an e-mail with agenda invite to a list of recipients.

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.utils import formatdate

from os.path import basename

from IPython.display import display, HTML

import pandas as pd
import smtplib
import time

In [ ]:
# Files
html_file = "" # HTML file
email_list = "" # CSV file with columns email, and firstname to personalize
agenda_invite = "" # ICS file

# Email account
mail_smtp = ""
mail_username = ""
mail_password = ""

# Email content
mail_from = "" # Email address
mail_subject = ""
mail_personalize = False

In [ ]:
# Open email content
mail_text_html = open(html_file, "r").read()
display(HTML(mail_text_html))

In [ ]:
# Open email list with addresses
email_addresses = pd.read_csv(email_list, sep=",")
recipients = email_addresses.to_dict('records')
len(recipients)

In [ ]:
# Connect to SMTP email server
s = smtplib.SMTP(mail_smtp)
s.login(mail_username, mail_password)

# Send email per recipient
for recipient in recipients:
    
    msg = MIMEMultipart()
    msg['From'] = mail_from
    msg['To'] = recipient['email']
    msg['Subject'] = mail_subject
    msg['Date'] = formatdate(localtime=True)
    
    # Personalize email 
    if mail_personalize:
        mime_text_html = MIMEText(''.join(mail_text_html.format(recipient['firstname'])), 'html')
    else:
        mime_text_html = MIMEText(''.join(mail_text_html), 'html')
    msg.attach(mime_text_html)
    
    # Add agenda invitation as attachment
    with open(agenda_invite, "rb") as fil:
        part = MIMEApplication(
            fil.read(),
            Name=basename(agenda_invite)
        )
        # After the file is closed
        part['Content-Disposition'] = 'attachment; filename="%s"' % basename(agenda_invite)
        msg.attach(part)
    
    s.sendmail(msg['From'], recipient['email'], msg.as_string())
    
    print("Email send to: {0}".format(msg['To']))
    time.sleep(5) # Break to prevent spam warnings
    
s.quit()